In [ ]:
---
title: "Chapter 5 - CV and Bootstrap"
author: "Dan"
date: "7 February 2018"
output: html_document
---

```{r setup, include=FALSE}
knitr::opts_chunk$set(echo = TRUE)
```
```{r}
set.seed(1)
library(ISLR)
data(Auto)

## Generally a good idea to set seed to a defined number when performing analysis such as cross validation that contains an element of randomness, so that they results obtained can be reproduced precisely at a later time ##

train <- sample(392,196) # creates a vector of 196 elements randomly sampled from 1 to 392.

lm.fit <- lm(mpg~horsepower, data=Auto, subset=train)
attach(Auto)
mean((mpg - predict(lm.fit, newdata=Auto))[-train]^2) # Test MSE

## The square of the mean of the difference between mpg and its predicted value only for the indexes that are not in the vector 'train' ##

## The [-train] must be OUTSIDE the predict call, we want the MSE calculated on the whole data set first, only then do we strip out the training set ##

## We can use newdata=Auto because we are stripping out the predictions of the training values later with [-train] so they never make it to the mean function
```

Estimated test MSE for the linear regression fit is 26.14. We can use the poly() function to estimate the test error for the quadratic and cubic regressions

```{r}
lm.fit2 <- lm(mpg~poly(horsepower ,2),data=Auto , subset=train)
mean((mpg - predict(lm.fit2, newdata = Auto))[- train]^2)

```

```{r}
lm.fit3 <- lm(mpg ~ poly(horsepower,3), data=Auto, subset=train)
mean((mpg-predict(lm.fit3, newdata = Auto))[-train]^2)
```

MSE for the quadratic fit  is 19.82 and for the cubic fit is 19.78

If we choose a different training set, we will obtain somewhat different errors on the validation set

```{r}
set.seed(2)
train <- sample (392,196) # a new sample

lm.fit <- lm(mpg~horsepower, data=Auto, subset=train)
mean((mpg-predict(lm.fit, newdata=Auto))[-train]^2)
```
```{r}
lm.fit2 <- lm(mpg~poly(horsepower,2), data=Auto, subset=train)
mean((mpg-predict(lm.fit2, newdata=Auto))[-train]^2)

lm.fit3 <- lm(mpg~poly(horsepower,3), data=Auto, subset=train)
mean((mpg-predict(lm.fit3, newdata=Auto))[-train]^2)
```

With a new training set we get a different answers for MSE for our models (23.3, 18.9, 19.26)

These results are consistent with our previous findings: a new model that predicts mpg using a quadratic function of horsepower performs better than a model that involves only a linear function of horsepower, and there is little evidence in favour of a model that uses a cubic function of horsepower

Leave-Out-One Cross-Validation

The LOOCV estimate can be automatically computed for any generalised linear model using the glm() and cv.glm() functions. In the lab for Chapter 4 we used the glm() function to perform basic logistic regression by passing in the family="binomial" argument. But if we use glm() to fit a model without passing the family artument, then it performs linear regression, just like lm(). For instance

```{r}
glm.fit <- glm(mpg~horsepower, data=Auto)
coef(glm.fit)
```

```{r}
lm.fit <- lm(mpg~horsepower, data=Auto)
coef(lm.fit)
```

produce identical linear regression models. In this lab, we will perform linear regression using the glm() function rather than the lm() function because the former can be used together with cv.glm(). The cv.glm() function is part of the 'boot' package

```{r}
library(boot)
glm.fit <- glm(mpg~horsepower, data=Auto)
cv.err <- cv.glm(glm.fit, data=Auto)
cv.err$delta
```

The cv.glm() function produces a list with several components. The two numbers in the delta vector contain the cross-validation results. In this case, the numbers are identical (up to two decimal places) and correspond to the LOOCV statistic given in (5.1) which is the mean of the mean squared errors. Below, we discuss a situation where the two numbers differ. Our cross-validation estimate for the test error is approaximately 24.23

We can repeat this procedure for increasingly complex polynomial fits. To automate the processs, we use the for() function to initiate a for loop which iteratively fits polynoial regressions for polynomials of order i=1 to i=5, computes the associated cross-validation error and stores it in the ith element of vector cv.error. We begin by initializing the vector. This command will likely take a couple of minutes to run.

```{r}

cv.error <- rep(0,5)

for (i in seq(1,5)) { ## gotta have dem brackets in the for call ##
glm.fit <- glm(mpg~poly(horsepower, i), data = Auto)
cv.error[i] <- cv.glm(Auto, glm.fit)$delta[1]
}

cv.error
```

As in figure 5.4, we see a sharp drop in estimated test MSE between the linear and the quadratic fits, but then no clear improvement from using higher order polynomials

k-fold Cross Validation

The cv.glm() function can also be used to implement k-fold CV. Below we use k=10, a common choice for k, on the Auto data set. We once again set a random seed and initialize a vector in which we will store the CV errors corresponding to the polynomial fits of order one to ten.

```{r}
set.seed(17)
cv.errors.10 <- rep(0,10)
for (i in 1:10) {
  glm.fit <- glm(mpg ~ poly(horsepower, i), data=Auto)
  cv.errors.10[i] <- cv.glm(glm.fit, data=Auto, K=10)$delta[1]
}

cv.errors.10
```

Notice that the computation time is much shorter for that of LOOCV. (In principle, the computation time for LOOCV for a least squares linear regression should be shorter than for k-fold CV due to the magic formula (5.2) but the cv.glm() function doesn't make use of it). We still see little evidence that using cubic or higher order polynomial terms leads to a lower test error than simply the quadratic fit

We saw in section 5.3.2 that the two numbers associated with delta are essentially the same when LOOCV is performed. When we instead perform k-fold CV, then the two numbers associated with delta differ slightly. The first is the standard k-fold CV estimate as in (5.3) The second is the bias-corrected version. On this data set, the two estimates are very similar to each other

#####
The Bootstrap
#####

We illustrate the use of a bootstrap in the simple example of section 5.2, as well as on an example involving estimating the accuracy of the linear regression model on the Auto data set.

#####
Estimating the Accuracy of a Statistic of Interest
#####

One of the great advantages of the bootstrap approach is that it can be applied in almost all situations. No complicated mathematical calculations are required. Performing a bootstrap analysis in R entails only two steps. 

First, we must create a function that computes the statistic of interest. Second, we use the boot() function, which is part of the boot library, to perform a bootstrap by repeatedly sampling observations from the data set with replacement.

The Portfolio data set in the ISLR package is described in Section 5.2. To illustrate the use of a bootstrap on this data, we must first create a function alpha.fn() which takes the input (X,Y) data as well as a vector indicating which observations should be used to estimate alpha. The function then outputs the estimate for alpha based on the selected observations.

```{r}
alpha.fn <- function (data, index) {
  X <- data$X[index]
  Y <- data$Y[index]
  return((var(Y)-cov(X,Y))/(var(X) + var(Y) - 2*cov(X,Y)))
}
```
This function returns an estimate for alpha based on applying (5.7) to the observations indexed by the argument index. For instance, the following code tells R to estimate alpha using all 100 observations

```{r}
set.seed(1)
alpha.fn(Portfolio, 1:100)
```
The next command uses the sample() function to randomly select 100 observations from the range 1 to 100, with replacement. This is equivalent to constructing the new bootstrap data set and recomputing alpha hat based on the new data set

```{r}
set.seed(1)
alpha.fn(Portfolio, sample(100,100,replace=T))
?sample
```

We can implement a bootstrap analysis by performing this command many times, recording all of the corresponding estimates for alpha, and computing the resulting standard deviation. However, the boot() function automates this aproach. Below we produce R=1000 bootstrap estimates for alpha

```{r}
boot(Portfolio, alpha.fn, R=1000)
```

The final output shows that using the original data, our estimate for alpha = 0.5758, and that the bootstrap estimate for standard error of our estimate of alpha is  0.0886

Estimating the Accuracy of a Linear Regression Model

The bootstrap approach can be used to assess the variability of the coefficient estimates and predictions from a statistical learning method. Here we use the bootstrap approach in order to assess the variability of the estimates for B0 and B1, the intercept and the slope term for the linear regression model that uses horsepower to predict mpg in the Auto data set. We will compare the estimates obtained using the bootstrap to those obtained using the formulas for SE(B^0) and SE(B^1) described in section 3.1.2

We first create a simple function, boot.fn() which takes the Auto data set as well as a set of indices for the observations, and returns the intercept and slope estimates for the linear regression model. We then apply this function to the full set of 392 observations in order to compute the estimates of B0 and B1 on the entire data set using the usual linear regression coefficient estimate formulas from Chapter 3. 
```{r}
boot.fn <- function (data, index) {
  return(coef(lm(mpg~horsepower, data=data, subset=index)))
}
boot.fn(Auto, 1:392)
```

The boot.fn() function can also be used in order to create bootstrap estimates for the intercept and slope terms by randomly sampling from among the observations with replacement. Here we give two examples:

```{r}
set.seed(1)
boot.fn(Auto, sample(392,392,replace=T))
```
```{r}
boot.fn(Auto, sample(392,392,replace=T))
```

Next, we use the boot() function to compute the standard errors of 1,000 bootstrap estimates for the intercept and slope terms

```{r}
boot(Auto, boot.fn, 1000)
```

This indicates that the bootstrap estimate for the standard error of B^0 is 0.86, and that the bootstrap estimate for the standard error of B^1 is 0.0074. As discussed in section 3.2.1, standard formulas can be used to compute the standard errors for the regression coefficients in a linear model. These can be obtained using the summary() function

```{r}
summary(lm(mpg~horsepower), data = Auto)$coef
```

The standard error estimates for B^0 and B^1 obtained using the formulas from section 3.1.2 are 0.717 for the intercept and 0.0064 for the slope. Interestingly, these are somewhat different from the estimates obtained using the bootstrap. Does this indicate a problem with the bootstrap? In fact, it suggests the opposite. Recall that the standard formulas given in Eqauation(3.8) rely on certain assumptions. For example, they depend on the unknown paramater Ï^2, the noise variance. We then estimate Ï^2 using the RSS. Now although the formula for the standard errors do not rely on the linear model being correct, the estimate for Ï^2 does. We see in figure 3.8 (page 105)  that there is a non-linear relationship in the data, and so the residuals from a linear fit will be inflated, and so will Ï^2. Secondly, the standard formulas assume (somewhat unrealistically) that the xi are fixed, and all the variability comes from the variation in the errors ei. The bootstrap approach does not rely on any of these assumptions, and so it is likely giving a more accurate estimate of the standard errors of B^0 and B^1 than is the summary() function.

Below, we compute the bootstrap standard error estimates and the standard linear regression estiamtes that result from fitting the quadratic model to the data. Since this model provides a good fit to the data (fig 3.8), there is now a better correspondence between the bootstrap estimates and the standard estimates of SE(B^0), SE(B^1) and SE(B^2).

```{r}
boot.fn <- function (data, index) {
   coef(lm(mpg~horsepower + I(horsepower^2)),data=data ,
subset=index)
}
set.seed(1)
boot(Auto, boot.fn, 1000)
```


```{r}

# P(jth observation is not in the bootstrap) = 1-(1/n)^n
# P(jth observations is in the bootstrap) = 1-(1-(1/n)^n)

# Plotting P(not in) for values of n 1 to 100,000
# Quickly P becomes ~64%

n <- seq(1,100000)
findprob <- function (n) {
  bootprob <- 1-(1-1/n)^n
  return (bootprob)
}
answers <- sapply(n,FUN = findprob)

mydf <- cbind(n,answers)
mydf <- as.data.frame(mydf)

library(ggplot2)
ggplot(data = mydf, aes(x = n, y = answers)) + geom_point() + ylab("probability") + xlab("n")
```

```{r}
store <- rep(NA,10000)
for(i in 1:10000) {
store[i]=sum(sample (1:100, rep=TRUE)==4) > 0
}
mean(store)

# we create 10,000 samples of 1:100 of n=100, each time the number 4 is included in one of our samples, the sum element increases by one, and if the sum element is > 0 the ith index of store returns TRUE. Here the mean of the logical vector store is 0.6344 so 63.44% of the samples contain a 4.
```
##########
Applied Exercises
##########

5a)

```{r}
data("Default")
set.seed(1)
attach(Default)
glm.fit <- glm(default ~ income+balance, data=Default, family="binomial")
dim(Default)

train <- sample(10000,5000)

glm.fit <- glm(default ~ income+balance, data=Default, subset=train, family="binomial")

glm.probs <- predict(glm.fit, newdata=Default[-train,])

glm.preds <- rep(0,5000)
glm.preds[glm.probs > 0.5] <- 1

table(glm.preds, Default$default[-train])
```

142/5000 misclassified = 2.84%

```{r}
# Now using 60/40 split for train/test

set.seed(5)
train <- sample(10000,5000)

glm.fit <- glm(default ~ income+balance, data=Default, subset=train, family="binomial")

glm.probs <- predict(glm.fit, newdata=Default[-train,])

glm.preds <- rep(0,5000)
glm.preds[glm.probs > 0.5] <- 1

table(glm.preds, Default$default[-train])
```

131/5000 misclassified = 2.62%

```{r}
set.seed(7)
train <- sample(10000,5000)

glm.fit <- glm(default ~ income+balance, data=Default, subset=train, family="binomial")

glm.probs <- predict(glm.fit, newdata=Default[-train,])

glm.preds <- rep(0,5000)
glm.preds[glm.probs > 0.5] <- 1

table(glm.preds, Default$default[-train])
```

135/5000 misclassified = 2.7% 

```{r}
set.seed(9)
train <- sample(10000,5000)

glm.fit <- glm(default ~ income+balance, data=Default, subset=train, family="binomial")

glm.probs <- predict(glm.fit, newdata=Default[-train,])

glm.preds <- rep(0,5000)
glm.preds[glm.probs > 0.5] <- 1

table(glm.preds, Default$default[-train])
```

137/5000 misclassified = 2.74%

Different result each time

```{r}
set.seed(2)
train <- sample(10000,5000)

glm.fit <- glm(default ~ income+balance+student, data=Default, subset=train, family="binomial")

glm.probs <- predict(glm.fit, newdata=Default[-train,])

glm.preds <- rep(0,5000)
glm.preds[glm.probs > 0.5] <- 1

table(glm.preds, Default$default[-train])
```

141/5000 misclassified = 2.82%

No significant reduction in test error rate

6)

```{r}
set.seed(1)

train <- sample(10000,5000)

glm.fit <- glm(default ~ income+balance, data=Default, subset=train, family="binomial")

summary(glm.fit)
```

Estimated standard error for income = 7.573e-06
Estimated standard error for balance = 3.467e-04

```{r}
boot.fn <- function (data, index) {
  coef(glm(default ~ income+balance, data=data, subset=index, family="binomial"))
}
boot(Default, boot.fn, 1000)
```

Estimated standard error for income = 4.59e-06
Estimated standard error for balance = 2.268e-04

Estimated standard errors from the bootstrap model are much smaller than from the glm() model. This is because the glm() model assumes that all of the variation in the fit comes from the error term (sigma^2 = Var(e)) so it requires that the model be a good fit. The bootstrap doesn't require this assumption so tends to give more accurate estimates of the standard error of our estimated coefficients.

7)

```{r}
set.seed(1)
data(Weekly)
attach(Weekly)
glm.fit <- glm(Direction ~ Lag1+Lag2, data=Weekly, family=binomial)

train <- Weekly[-1,]
test <- Weekly[1,]

glm.fit2 <- glm(Direction ~ Lag1+Lag2, data=train, family=binomial)
glm.probs <- predict(glm.fit2, newdata = test)

glm.preds <- "Down"
glm.preds[glm.probs > 0.5] <- "Up"

table(glm.preds, Weekly$Direction[1])

# Model predicts Down. This was correctly classified.

errors.vec <- rep(0,1089)

for (i in 1:1089) {
  training.set <- Weekly[-i,]
  glm.fit <- glm(Direction~Lag1+Lag2, data=training.set, family = binomial)
  glm.prob <- predict(glm.fit, newdata=Weekly[i,])
  glm.pred <- "Down"
  glm.pred[glm.prob > 0.5] <- "Up"
  if (glm.pred == Weekly$Direction[i]) {errors.vec[i] <- 1}
}

mean(errors.vec)

# LOOCV Test error rate is 45.45%!
# There is likely to be high variance in the test error rate because the n LOOCV samples are highly positively correlated

```

8)

```{r}
set.seed(1)
x=rnorm(100)
y= x-2*x^2+rnorm(100)

```
n=100
p=1

```{r}
plot(x,y)
```
PRETTY RAINBOW!

```{r}
####
## Lost all work from this question, see answers
####

####
#9a)
####

library(MASS)
data(Boston)
attach(Boston)

medv.mu <- mean(medv)
medv.mu
```

```{r}
####
#b)#
####

std.err.mu <- sd(medv) / sqrt(nrow(Boston))
std.err.mu
```

```{r}
####
#c)#
####

library(boot)
set.seed(1)

mean.fn <- function(var, id) {
  return(mean(var[id]))
}

boot.res <- boot(Boston$medv,mean.fn,100)
boot.res

```
```{r}
## Estimate from bootstrap = 0.435, reasonably close to 0.41

## 95% CI for medv = medv.mu +/- 2*SE = [21.663,23.403] ##

t.test(Boston$medv) # very similar results, t-test CI is slightly tighter
```

```{r}
median.mu <- median(Boston$medv)
median.mu
```

```{r}
median.fn <- function (var, id) {
  return(median(var[id]))
}

boot(Boston$medv, median.fn, 100)
```

```{r}
## Standard error of the median using the bootstrap method = 0.39 ## Very close to the standard error of the mean which makes sense ##

quantile.10.estimate <- quantile (Boston$medv, probs=0.1)

```

```{r}
tenthquantile.estimate <- function (var,id) {
  return(quantile(var[id], probs=0.1))
}

boot(Boston$medv, tenthquantile.estimate, 100)
```

```{r}
## Estimate for standard error of the 10th quantile of the population = 0.512
```


